<a href="https://colab.research.google.com/github/manashpratim/Bosch-Summer-Internship/blob/master/VideoV1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Data is available at https://drive.google.com/drive/folders/1NFYIaXjL8V5kvZo3g9JEafLQ3scslWic?usp=sharing

### This notebook is for video (feature) extraction for the models in VideoV2 notebook

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip -q '/content/drive/My Drive/mosi_data/mosi.zip'

In [ ]:
!pip install mtcnn
!pip install git+https://github.com/rcmalli/keras-vggface.git

In [ ]:
import keras_vggface
from mtcnn.mtcnn import MTCNN
import numpy as np 
import cv2
import pandas as pd
from matplotlib import pyplot
from PIL import Image
from keras_vggface.vggface import VGGFace
model = VGGFace(model='vgg16', include_top=False, input_shape=(224, 224, 3), pooling='avg')
from keras_vggface.utils import preprocess_input

In [ ]:
def extract_face(pixels):
    detector = MTCNN()
    results = detector.detect_faces(pixels)
    if results:
      x1, y1, width, height = results[0]['box']
      x2, y2 = x1 + width, y1 + height
      face = pixels[y1:y2, x1:x2]
    else:
      face=pixels
    try:
      image = Image.fromarray(face)
      image = image.resize((100,100))
      face = np.array(image)/255.
      return face.astype('float32')
    except:
      return []

In [ ]:
def get_embeddings(samples):
	#samples = preprocess_input(samples, version=2)
	return model.predict(samples).astype('float32')

In [ ]:
#For Model 1
def extract_face_from_video1(videofile,video_dims=None,framerate=0.5):
    vidcap = cv2.VideoCapture(videofile)
    def getFrame(sec):
        vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
        hasFrames,image = vidcap.read()
        if hasFrames:
            if video_dims:
              image =  cv2.resize(image, video_dims)
            return image
        else:
          return []

    sec = 0
    framerate = framerate
    frames = []
    while True:
        sec = sec + framerate
        sec = round(sec, 2)
        im = getFrame(sec)
        if len(im)>0:
          im = extract_face(im[...,::-1])   
          frames.append(im)
        else:
          break
    return np.array(frames).astype('float32')

In [ ]:
#For Model 2 
ef extract_face_from_video2(videofile,video_dims=None,framerate=0.1):
    vidcap = cv2.VideoCapture(videofile)
    def getFrame(sec):
        vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
        hasFrames,image = vidcap.read()
        if hasFrames:
            if video_dims:
              image =  cv2.resize(image, video_dims)
            return image
        else:
          return []

    sec = 0
    framerate = framerate
    frames = []
    while True:
        sec = sec + framerate
        sec = round(sec, 2)
        im = getFrame(sec)
        if len(im)>0:
          im = (im[...,::-1]/255.).astype('float32')
          frames.append(im)
        else:
          break

    return np.array(frames).astype('float32')

In [ ]:
#For Model 3
def extract_face_from_video3(videofile,video_dims=None,framerate=0.03):
    vidcap = cv2.VideoCapture(videofile)
    def getFrame(sec):
        vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
        hasFrames,image = vidcap.read()
        if hasFrames:
            if video_dims:
              image =  cv2.resize(image, video_dims)
            return image
        else:
          return []

    sec = 0
    framerate = framerate
    frames = []
    while True:
        sec = sec + framerate
        sec = round(sec, 2)
        im = getFrame(sec)
        if len(im)>0:
          im = (im[...,::-1]/255.).astype('float32')        
          frames.append(im)
        else:
          break
    return get_embeddings(np.array(frames))

In [ ]:
def plot(img):
  pyplot.imshow(img)
  pyplot.show()

In [ ]:
#For Model 1
def visual_features1(mypath,files):
    video_dic = {}
    for file in files:
      videofile = mypath+'/'+file+'.mp4'
      samples = extract_face_from_video1(videofile)
      video_dic[file] = get_embeddings(samples)
      del samples
    return video_dic

In [ ]:
#For Model 2
def visual_features2(mypath,files):
    video_dic = {}
    for file in files:
      videofile = mypath+'/'+file+'.mp4'
      video_dic[file] = extract_face_from_video2(videofile,(64,64))
      
    return video_dic

In [ ]:
#For Model 3
def visual_features3(mypath,files):
    video_dic = {}
    for file in files:
      videofile = mypath+'/'+file+'.mp4'
      video_dic[file] = extract_face_from_video3(videofile,(224,224))
      
    return video_dic

In [ ]:
def get_file_names(mypath,savefile):
  from os import listdir
  from os.path import isfile, join
  onlyfiles = [f[:f.find('.')] for f in listdir(mypath) if isfile(join(mypath, f))]
  with open(savefile, 'w') as f:
    for item in onlyfiles:
        f.write(item)
        f.write('\n')
  return onlyfiles

In [ ]:
mypath = '/content/Raw/Video/Segmented'
files = get_file_names(mypath,'mosi_files.txt')
video_dic = visual_features3(mypath,files)

In [ ]:
# Data Format Alignment
dic1 = {}
for file in files:
  new = file[:-file[::-1].find('_')-1]
  dic1[file] = new

newdic = {}
newdic1 = {}
for key in dic1:
  newkey = dic1[key]
  if newkey not in newdic:
    newdic[newkey] = {}
  if newkey not in newdic1:
    newdic1[newkey] = {}
  newdic1[newkey][key] = video_dic[key]
  k = key[-key[::-1].find('_'):]
  newdic[newkey][int(k)] = video_dic[key]

nd = {}
for key in newdic:
    nd[key] = []
    for k in sorted(newdic[key].keys()):
      nd[key].append(newdic[key][k])

In [ ]:
import pickle
with open('/content/drive/My Drive/mosi_data/video_frames_dense.pickle', 'wb') as handle:
    pickle.dump(nd, handle, protocol=pickle.HIGHEST_PROTOCOL)